                                                    CLASSIFICATION IN MACHINE LEARNING USING
                                                    i)logistic regression(sklearn)
                                                    ii)neural network
                                                    iii)random forest classifier
                                                    

IMPORTING RELEVANT LIBRARIES

In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression




In [7]:
df_preprocessed=pd.read_csv("Absenteeism_preprocessed.csv")

In [5]:
df_preprocessed

,reason1,reason2,reaon3,reason4,month,week_day,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,2,1.006,0.413,-0.536,-0.806,0.767,1,2,1,4
1,0,0,0,0,7,2,-1.575,-1.142,2.131,-0.806,1.003,1,1,0,0
2,0,0,0,1,7,3,-0.654,1.427,0.248,-0.806,1.003,1,0,0,2
3,1,0,0,1,7,4,0.855,-1.683,0.405,-0.806,-0.644,1,2,0,4
4,0,0,0,1,7,4,1.006,0.413,-0.536,-0.806,0.767,1,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,1,5,3,-0.654,-0.534,0.562,-0.854,-1.114,0,2,0,8
696,1,0,0,1,5,3,0.040,-0.263,-1.320,-0.854,-0.644,1,1,2,3
697,1,0,0,1,5,4,1.625,-0.939,-1.320,-0.854,-0.409,0,0,0,8
698,0,0,0,1,5,4,0.191,-0.939,-0.693,-0.854,-0.409,0,0,0,2


In [9]:
df_preprocessed['Targets']=np.where(df_preprocessed["Absenteeism Time in Hours"]>np.median(df_preprocessed['Absenteeism Time in Hours']),1,0) #define categorical target

df_preprocessed=df_preprocessed.drop("Absenteeism Time in Hours",axis=1) #drop original target as we are only concerned with the binary outcomes


inputs=df_preprocessed.iloc[:,0:14] 

print(df_preprocessed)


     reason1  reason2  reaon3  reason4  month  week_day  \
0          0        0       0        1      7         2   
1          0        0       0        0      7         2   
2          0        0       0        1      7         3   
3          1        0       0        1      7         4   
4          0        0       0        1      7         4   
..       ...      ...     ...      ...    ...       ...   
695        1        0       0        1      5         3   
696        1        0       0        1      5         3   
697        1        0       0        1      5         4   
698        0        0       0        1      5         4   
699        0        0       0        1      5         4   

     Transportation Expense  Distance to Work    Age  Daily Work Load Average  \
0                     1.006             0.413 -0.536                   -0.806   
1                    -1.575            -1.142  2.131                   -0.806   
2                    -0.654             1.427  0

In [10]:

targets=df_preprocessed['Targets']
targets=np.array(targets)
np.unique(targets)

array([0, 1])

In [12]:
print(inputs.shape)
print(targets.shape)

(700, 14)
(700,)


Split the datset into training and testing part

In [13]:
x_train,x_test,y_train,y_test=train_test_split(inputs,targets,train_size=0.85,random_state=10)
print(x_test.shape,x_train.shape ,"\n",y_test.shape,y_train.shape)
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,train_size=0.9,random_state=10)
print(x_val.shape,x_train.shape ,"\n",y_val.shape,y_train.shape)

(105, 14) (595, 14) 
 (105,) (595,)
(60, 14) (535, 14) 
 (60,) (535,)


I. USING LOGISTIC REGRESSION 

In [15]:
reg=LogisticRegression()
reg.fit(x_train,y_train)
print("score using builtin function:",reg.score(x_test,y_test))
print("\nchecking manually")
predicted=reg.predict(x_test)
accuracy_matrix=np.array([int(val) for val in predicted==y_test])
print("\t\t\t\taccuracy_matrix\n",accuracy_matrix)
print("\nscore:",sum(accuracy_matrix)/accuracy_matrix.shape[0])

score using builtin function: 0.7523809523809524

checking manually
				accuracy_matrix
 [0 1 0 0 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 0
 1 0 0 1 1 0 0 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 1]

score: 0.7523809523809524


C:\Users\HP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [88]:
features=inputs.columns.values
coefficents=reg.coef_[0]

df_reg=pd.DataFrame({"features": features,"coef":coefficents})
df_reg=pd.concat([df_reg,pd.DataFrame({"features":"intercept","coef":reg.intercept_})],axis=0)
df_reg['abs_coef']=df_reg['coef'].abs()
df_reg=df_reg.sort_values(by="abs_coef",ascending=False)
df_reg=df_reg.drop('abs_coef',axis=1)
df_reg


,features,coef
0,intercept,-3.305284
3,reason4,2.671289
2,reaon3,2.007978
0,reason1,1.901151
6,Transportation Expense,0.617478
12,Children,0.429931
13,Pets,-0.326842
8,Age,-0.256288
10,Body Mass Index,0.195481
1,reason2,0.189428


II.USING NEURAL NETWORK

In [16]:
import tensorflow


In [17]:
input_size=14
hidden_size=100
output_size=1


model=tensorflow.keras.Sequential([
    tensorflow.keras.layers.Dense(hidden_size,activation='tanh'),
    tensorflow.keras.layers.Dense(hidden_size,activation='tanh'),
    
    
    tensorflow.keras.layers.Dense(output_size,activation='sigmoid')])
optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.9,beta_2=0.999)
model.compile(loss='binary_crossentropy',optimizer=tensorflow.keras.optimizers.Adam(),metrics='accuracy')


In [18]:

batch_size = 100

max_epochs = 100

# setting  an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tensorflow.keras.callbacks.EarlyStopping(patience=2)

# fit the model

model.fit(x_train, # train inputs
          y_train, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          callbacks=[early_stopping], # early stopping 
          validation_data=(x_val, y_val), # validation data
          #verbose = 1 # making sure we get enough information about the training process
          )  

Epoch 1/100
6/6 [==============================] - 2s 69ms/step - loss: 0.6870 - accuracy: 0.5738 - val_loss: 0.6211 - val_accuracy: 0.6667
Epoch 2/100
6/6 [==============================] - 0s 10ms/step - loss: 0.6492 - accuracy: 0.6056 - val_loss: 0.5838 - val_accuracy: 0.7000
Epoch 3/100
6/6 [==============================] - 0s 11ms/step - loss: 0.6283 - accuracy: 0.6673 - val_loss: 0.5668 - val_accuracy: 0.7667
Epoch 4/100
6/6 [==============================] - 0s 17ms/step - loss: 0.6101 - accuracy: 0.6393 - val_loss: 0.5530 - val_accuracy: 0.7000
Epoch 5/100
6/6 [==============================] - 0s 18ms/step - loss: 0.5999 - accuracy: 0.6748 - val_loss: 0.5434 - val_accuracy: 0.7167
Epoch 6/100
6/6 [==============================] - 0s 16ms/step - loss: 0.5858 - accuracy: 0.6972 - val_loss: 0.5422 - val_accuracy: 0.7167
Epoch 7/100
6/6 [==============================] - 0s 13ms/step - loss: 0.5793 - accuracy: 0.7084 - val_loss: 0.5349 - val_accuracy: 0.7167
Epoch 8/100
6/6 [===

In [19]:
loss,accuracy=model.evaluate(x_test,y_test)
print("accuracy =",accuracy)

4/4 [==============================] - 0s 3ms/step - loss: 0.5188 - accuracy: 0.7333
accuracy = 0.7333333492279053


III.USING RANDOM FOREST CLASSIFIER

In [23]:
from sklearn.ensemble import RandomForestClassifier


In [24]:
clf=RandomForestClassifier(n_estimators=100,random_state=42)
model=clf.fit(x_train,y_train)

In [26]:
outputs=model.predict(x_test)
#print(outputs)
accuracy_matrix=outputs==y_test
print(np.array([int(value) for value in accuracy_matrix]))

accuracy=sum(accuracy_matrix)/len(accuracy_matrix)
print(accuracy)

[0 1 0 0 1 0 1 0 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0
 1 0 0 1 1 0 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 1]
0.780952380952381
